<a href="https://colab.research.google.com/github/Adarshkumar61/Deep_Learning_using_colab/blob/main/VGG_transfer_learning_CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import os
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam

In [4]:
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d alxmamaev/flowers-recognition
!unzip -q flowers-recognition.zip -d /content/

Saving kaggle.json to kaggle (2).json
Dataset URL: https://www.kaggle.com/datasets/alxmamaev/flowers-recognition
License(s): unknown
 96% 216M/225M [00:00<00:00, 741MB/s] 
100% 225M/225M [00:00<00:00, 776MB/s]


In [11]:
img_size = 224
batch_size = 32
epochs = 10

IMAGEDATAFGENERATOR AND DATA SPLIT

In [9]:
datagen = ImageDataGenerator(
    rescale= 1./255,
    validation_split = 0.2,
    shear_range = 0.2,
    zoom_range = 0.1,
    horizontal_flip = True,
    rotation_range = 15
)

In [12]:
train_gen = datagen.flow_from_directory(
    '/content/flowers',
    target_size = (img_size, img_size),
    batch_size = batch_size,
    class_mode = 'categorical',
    subset = 'training'
)

Found 3457 images belonging to 5 classes.


In [14]:
val_gen = datagen.flow_from_directory(
    '/content/flowers',
    target_size = (img_size, img_size),
    batch_size = batch_size,
    class_mode = 'categorical',
    subset = 'validation'
)

Found 860 images belonging to 5 classes.


LOADING VGG16 AND CUSTOMIZING TOP LAYER

In [15]:
vgg = VGG16(include_top=False, weights='imagenet', input_shape=(img_size, img_size, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [16]:
for layer in vgg.layers:
  layer.trainable = False

creating Dense Layer

In [18]:
x = Flatten()(vgg.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(train_gen.num_classes, activation='softmax')(x)

model = Model(inputs = vgg.input, outputs = output)

MODEL COMPILE AND RUN

In [20]:
model.compile(
    optimizer = Adam(),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [21]:
model.fit(train_gen, epochs = epochs, validation_data= val_gen)
# var name will be history

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 87s 665ms/step - accuracy: 0.4207 - loss: 2.0083 - val_accuracy: 0.7384 - val_loss: 0.7808
Epoch 2/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 57s 527ms/step - accuracy: 0.6299 - loss: 0.9308 - val_accuracy: 0.7442 - val_loss: 0.7821
Epoch 3/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 82s 524ms/step - accuracy: 0.6346 - loss: 0.8722 - val_accuracy: 0.7558 - val_loss: 0.6890
Epoch 4/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 58s 530ms/step - accuracy: 0.6417 - loss: 0.8616 - val_accuracy: 0.7779 - val_loss: 0.6381
Epoch 5/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 58s 532ms/step - accuracy: 0.6770 - loss: 0.7753 - val_accuracy: 0.7872 - val_loss: 0.5944
Epoch 6/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 82s 529ms/step - accuracy: 0.7020 - loss: 0.7068 - val_accuracy: 0.7523 - val_loss: 0.6324
Epoch 7/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 60s 550ms/step - accuracy: 0.7061 - loss: 0.6715 - val_accuracy: 0.7895 - val_loss: 0.5882
Epoch 8/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 62s 565ms/step - accuracy: 0.7075 - loss: 0

In [22]:
loss, acc = model.evaluate(val_gen)
print(f'Loss: {loss}, Accuracy: {acc}')

27/27 ━━━━━━━━━━━━━━━━━━━━ 12s 419ms/step - accuracy: 0.7886 - loss: 0.5873
Loss: 0.5685439109802246, Accuracy: 0.789534866809845


PREDICTION

In [47]:
img_path = '/content/flowers/sunflower/12471791574_bb1be83df4.jpg'
img = image.load_img(img_path, target_size=(img_size, img_size))
img_array = image.img_to_array(img)/ 255.0
img_array = np.expand_dims(img_array, axis=0)


In [48]:
prediciton = model.predict(img_array)
predict_class = np.argmax(prediciton[0])
label = list(train_gen.class_indices.keys())
print(f'predicted class is {label[predict_class]}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
predicted class is sunflower


In [49]:
import os
actual_class = os.path.basename(os.path.dirname(img_path))
print(f'The actual class is {actual_class}')

The actual class is sunflower


In [ ]:
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.show()